In [ ]:
!pip install torch torchvision --quiet

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
seed = 42

transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5),
                         (0.5, 0.5, 0.5)),
])

full_trainset = torchvision.datasets.CIFAR10(
    root="./data",
    train=True,
    download=True,
    transform=transform,
)

# 고정된 랜덤시드로 split
train_size = int(0.9 * len(full_trainset))
val_size = len(full_trainset) - train_size

trainset, validset = torch.utils.data.random_split(
    full_trainset,
    [train_size, val_size],
    generator=torch.Generator().manual_seed(seed)  # key point!
)

cuda


100%|██████████| 170M/170M [00:04<00:00, 41.1MB/s]


In [ ]:
class MyNet(nn.Module):
    def __init__(self, num_classes=10):
        super(MyNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, 11, stride=4, padding=2),
            nn.ReLU(True),
            nn.MaxPool2d(3, stride=2),

            nn.Conv2d(64, 192, 5, padding=2),
            nn.ReLU(True),
            nn.MaxPool2d(3, stride=2),

            nn.Conv2d(192, 384, 3, padding=1),
            nn.ReLU(True),
            nn.Conv2d(384, 256, 3, padding=1),
            nn.ReLU(True),
            nn.Conv2d(256, 256, 3, padding=1),
            nn.ReLU(True),
            nn.MaxPool2d(3, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        return self.classifier(x)


In [ ]:
def train(model, loader, optimizer, criterion):
    model.train()
    loss_sum, correct, total = 0, 0, 0

    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        loss_sum += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    return loss_sum / len(loader), 100 * correct / total


def test(model, loader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    return 100 * correct / total


In [ ]:
trainloader = DataLoader(trainset, batch_size=128, shuffle=True)
validloader = DataLoader(validset, batch_size=128, shuffle=False)

In [ ]:
model = MyNet().to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

best_val_acc = 0.0

for epoch in range(2):
    loss, train_acc = train(model, trainloader, optimizer, criterion)
    val_acc = test(model, validloader)

    print(f"[Epoch {epoch+1}] Loss: {loss:.4f} | Train Acc: {train_acc:.2f}% | Valid Acc: {val_acc:.2f}%")

    # Validation 성능이 좋아질 때마다 모델 저장
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "my_model.pt")
        print(f"  --> Saved best model (Val Acc = {val_acc:.2f}%)")

print("Training finished.")
print("Best model saved to my_model.pt")



[Epoch 1] Loss: 1.6814 | Train Acc: 37.78% | Valid Acc: 49.96%
  --> Saved best model (Val Acc = 49.96%)
[Epoch 2] Loss: 1.2132 | Train Acc: 56.22% | Valid Acc: 60.50%
  --> Saved best model (Val Acc = 60.50%)
Training finished.
Best model saved to my_model.pt
